In [1]:
import pandas as pd

In [2]:
files = ['../Data/Commodities by visitors/CommodityByVisitors2018.csv', 
         '../Data/Commodities by visitors/CommodityByVisitors2019.csv', 
         '../Data/Commodities by visitors/CommodityByVisitors2020.csv', 
         '../Data/Commodities by visitors/CommodityByVisitors2021.csv', 
         '../Data/Commodities by visitors/CommodityByVisitors2022.csv', 
         '../Data/Commodities by visitors/CommodityByVisitors2023.csv']

In [3]:
dfs = []
for file in files:
    df = pd.read_csv(file)
    year = int(file.split('CommodityByVisitors')[1].split('.csv')[0])
    df['Year']= year
    dfs.append(df)

In [4]:
visitor_type_merged_df = pd.concat(dfs, ignore_index=True)

In [5]:
visitor_type_merged_df.to_csv('../CommodityByVisitors.csv', index=False)

In [6]:
visitor_type_merged_df.head()

,Commodity,Total consumption,Resident households,Business,Government,Nonresidents,Total tourism demand,Nontourism demand,Tourism commodity ratio,Year
0,Traveler accommodations,"228,327","109,070","71,631","6,568","41,058","228,327",0,1.00,2018
1,Food and beverage services,"998,014","84,117","40,934","4,766","24,252","154,069","843,944",0.15,2018
2,Domestic passenger air transportation services,"167,622","103,239","48,734","15,620",29,"167,622",0,1.00,2018
3,International passenger air transportation ser...,"102,623","35,571","24,965","1,625","40,462","102,623",0,1.00,2018
4,Passenger rail transportation services,"2,403","1,193",966,244,0,"2,403",0,1.00,2018


In [7]:
visitor_type_merged_df.to_csv('commodities_by_visitor_types.csv')

In [8]:
melted_df = pd.melt(visitor_type_merged_df,
                    id_vars=['Commodity', 'Total consumption', 'Total tourism demand',
                             'Nontourism demand', 'Tourism commodity ratio', 'Year'],
                    value_vars=['Resident households', 'Business', 'Government', 'Nonresidents'],
                    var_name='Visitor Type',
                    value_name='Demand')

melted_df.head()

,Commodity,Total consumption,Total tourism demand,Nontourism demand,Tourism commodity ratio,Year,Visitor Type,Demand
0,Traveler accommodations,"228,327","228,327",0,1.00,2018,Resident households,"109,070"
1,Food and beverage services,"998,014","154,069","843,944",0.15,2018,Resident households,"84,117"
2,Domestic passenger air transportation services,"167,622","167,622",0,1.00,2018,Resident households,"103,239"
3,International passenger air transportation ser...,"102,623","102,623",0,1.00,2018,Resident households,"35,571"
4,Passenger rail transportation services,"2,403","2,403",0,1.00,2018,Resident households,"1,193"


melted_df.to_csv('commodities_by_visitor_melted.csv')

Clean columns

In [9]:
clean = [
    'Total consumption', 'Resident households', 'Business', 'Government',
    'Nonresidents', 'Total tourism demand', 'Nontourism demand']

for col in clean:
    visitor_type_merged_df[col] = visitor_type_merged_df[col].replace(',', '', regex=True).astype(float)

Combine Business and Government column together

In [10]:
visitor_type_merged_df['Business and Government'] = visitor_type_merged_df['Business'] + visitor_type_merged_df['Government']

Combine residents and non residents column together

In [11]:
visitor_type_merged_df['Resident and Nonresident'] = visitor_type_merged_df['Resident households'] + visitor_type_merged_df['Nonresidents']

In [12]:
visitor_type_merged_df_filtered = visitor_type_merged_df[visitor_type_merged_df['Year'].between(2018, 2023)]

yearly_sums = visitor_type_merged_df_filtered.groupby('Year')[
    ['Total consumption','Business and Government', 'Resident and Nonresident', 'Total tourism demand', 'Nontourism demand']].sum().reset_index()

yearly_sums

,Year,Total consumption,Business and Government,Resident and Nonresident,Total tourism demand,Nontourism demand
0,2018,39248374.0,360850.0,998794.0,1359646.0,37888726.0
1,2019,40361847.0,385502.0,1036385.0,1421886.0,38939963.0
2,2020,39221150.0,192827.0,470376.0,663200.0,38557950.0
3,2021,44846646.0,348659.0,768569.0,1117227.0,43729417.0
4,2022,50063247.0,427031.0,1151463.0,1578492.0,48484754.0
5,2023,51782020.0,454078.0,1312895.0,1766977.0,50015043.0


Top 5 by total tourism demand

In [13]:
top5_commodities = visitor_type_merged_df.sort_values('Total tourism demand', ascending=False).head(5)
top5_commodities[['Commodity', 'Total tourism demand']]

,Commodity,Total tourism demand
130,Traveler accommodations,283099.0
153,Shopping,271004.0
104,Traveler accommodations,254083.0
131,Food and beverage services,249998.0
127,Shopping,238368.0


Consumption by Business and Government category

In [14]:
consumption_BAG = visitor_type_merged_df.groupby('Year')['Business and Government'].sum()
consumption_BAG

Year
2018    360850.0
2019    385502.0
2020    192827.0
2021    348659.0
2022    427031.0
2023    454078.0
Name: Business and Government, dtype: float64

Consumption by Resident and Nonresident category

In [15]:
consumption_RAN = visitor_type_merged_df.groupby('Year')['Resident and Nonresident'].sum()
consumption_RAN

Year
2018     998794.0
2019    1036385.0
2020     470376.0
2021     768569.0
2022    1151463.0
2023    1312895.0
Name: Resident and Nonresident, dtype: float64

Top 5 by Business and Government category

In [16]:
consumption_by_commodity = visitor_type_merged_df.groupby('Commodity')['Business and Government'].sum()
top5_consumption = consumption_by_commodity.sort_values(ascending=False).head()
top5_consumption

Commodity
Traveler accommodations                           478763.0
Food and beverage services                        293714.0
Domestic passenger air transportation services    282388.0
Travel arrangement and reservation services       195447.0
Shopping                                          190916.0
Name: Business and Government, dtype: float64

Top 5 by Resident and Nonresident category

In [17]:
consumption_by_commodity = visitor_type_merged_df.groupby('Commodity')['Resident and Nonresident'].sum()
top5_consumption = consumption_by_commodity.sort_values(ascending=False).head()
top5_consumption

Commodity
Shopping                                          889921.0
Traveler accommodations                           861584.0
Food and beverage services                        729120.0
Domestic passenger air transportation services    686299.0
Gasoline                                          667329.0
Name: Resident and Nonresident, dtype: float64

In [19]:
visitor_type_merged_df.to_csv('visitorstypebycommodity.csv')